# Anusha's Final Term Project
->KNN,SVM,Random Forest and LSTM 


In [7]:
# Importing the important libraries

import numpy as np
import random
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix, accuracy_score, f1_score

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier # Random Forest Classifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

from keras.models import Sequential



import warnings
warnings.filterwarnings('ignore')

In [9]:
# Loading the dataset

df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [10]:
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
df['Sex']= label_encoder.fit_transform(df['Sex'])
df['ChestPainType']= label_encoder.fit_transform(df['ChestPainType'])
df['RestingECG']= label_encoder.fit_transform(df['RestingECG'])
df['ExerciseAngina']= label_encoder.fit_transform(df['ExerciseAngina'])
df['ST_Slope']= label_encoder.fit_transform(df['ST_Slope'])

 
df['ST_Slope'].unique()

array([2, 1, 0])

In [11]:
print(df.shape)
print(df["HeartDisease"].value_counts())

(918, 12)
1    508
0    410
Name: HeartDisease, dtype: int64


In [12]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [13]:
df.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,0.789760,0.781046,132.396514,198.799564,0.233115,0.989107,136.809368,0.404139,0.887364,1.361656,0.553377
std,9.432617,0.407701,0.956519,18.514154,109.384145,0.423046,0.631671,25.460334,0.490992,1.066570,0.607056,0.497414
min,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.000000,0.000000,-2.600000,0.000000,0.000000
25%,47.000000,1.000000,0.000000,120.000000,173.250000,0.000000,1.000000,120.000000,0.000000,0.000000,1.000000,0.000000
50%,54.000000,1.000000,0.000000,130.000000,223.000000,0.000000,1.000000,138.000000,0.000000,0.600000,1.000000,1.000000
75%,60.000000,1.000000,2.000000,140.000000,267.000000,0.000000,1.000000,156.000000,1.000000,1.500000,2.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,1.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    int32  
 2   ChestPainType   918 non-null    int32  
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    int32  
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    int32  
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    int32  
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int32(5), int64(6)
memory usage: 68.3 KB


In [15]:
# Seperating the dependent and the independent columns

X = df.drop("HeartDisease" , axis = 1)
y = df['HeartDisease']

In [16]:
# Seperating the train and test dataset
x_train,x_test,y_train,y_test = train_test_split(X,y,stratify = y , random_state=42,test_size=0.2)

In [17]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(734, 11)
(734,)
(184, 11)
(184,)


In [18]:
x_train[:2]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
485,63,1,1,139,217,1,2,128,1,1.2,1
486,55,1,1,110,214,1,2,180,0,0.4,2


# KNN Algorithm

In [19]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(x_train,y_train)
y_pred_knn = knn.predict(x_test)

print(knn.score(x_test,y_test))
print(classification_report(y_test,y_pred_knn))
print(confusion_matrix(y_test,y_pred_knn))

0.6956521739130435
              precision    recall  f1-score   support

           0       0.67      0.62      0.65        82
           1       0.71      0.75      0.73       102

    accuracy                           0.70       184
   macro avg       0.69      0.69      0.69       184
weighted avg       0.69      0.70      0.69       184

[[51 31]
 [25 77]]


In [20]:
type(y_test)

pandas.core.series.Series

# SVM Algorithm

In [21]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state=27)
classifier.fit(x_train, y_train)


SVC(random_state=27)

In [22]:
y_pred_svm = classifier.predict(x_test)

In [23]:
print(classifier.score(x_test,y_test))
print(classification_report(y_test, y_pred_svm))
print(confusion_matrix(y_test,y_pred_svm))

0.717391304347826
              precision    recall  f1-score   support

           0       0.68      0.70      0.69        82
           1       0.75      0.74      0.74       102

    accuracy                           0.72       184
   macro avg       0.71      0.72      0.71       184
weighted avg       0.72      0.72      0.72       184

[[57 25]
 [27 75]]


# Random Forest Algorithm

In [24]:
randomforest = RandomForestClassifier()
randomforest.fit(x_train,y_train)
y_pred_rf = randomforest.predict(x_test)

print('Train Accuracy %s' % round(accuracy_score(y_test, y_pred_rf),2))
print('Train F1-score %s' % f1_score(y_test, y_pred_rf, average=None))
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:\n",confusion_matrix(y_test, y_pred_rf))

# LSTM - Long Short Term Memory


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

from keras.callbacks import EarlyStopping
import math

In [33]:
X_train,Y_train,X_test,Y_test=train_test_split(X,y,test_size=0.5,random_state=0)

In [34]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(459, 11)
(459, 11)
(459,)
(459,)


In [35]:
print(X_train[:1])

    Age  Sex  ChestPainType  RestingBP  Cholesterol  FastingBS  RestingECG  \
46   37    1              0        120          223          0           1   

    MaxHR  ExerciseAngina  Oldpeak  ST_Slope  
46    168               0      0.0         2  


In [36]:
X_train=np.expand_dims(X_train, axis=2) 
Y_train=np.expand_dims(Y_train, axis=2)
es=EarlyStopping(patience=7)
model=Sequential()
model.add(LSTM(1,input_shape=(11,1)))
model.add(Dense(1,activation='softmax'))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train,X_test,epochs=10,batch_size=1,verbose=1,callbacks=[es])
predict = model.predict(Y_train)
c = model.evaluate(Y_train,Y_test)

Epoch 1/10
459/459 [==============================] - 5s 6ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 2/10
459/459 [==============================] - 3s 6ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 3/10
459/459 [==============================] - 2s 4ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 4/10
459/459 [==============================] - 3s 6ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 5/10
459/459 [==============================] - 2s 4ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 6/10
459/459 [==============================] - 3s 7ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 7/10
459/459 [==============================] - 3s 6ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 8/10
459/459 [==============================] - 3s 6ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 9/10
459/459 [==============================] - 3s 6ms/step - loss: 0.7686 - accuracy: 0.5447
Epoch 10/10
15/15 [==============================] - 1s 8ms/step - loss: 0.7512 - accuracy: 0.5621


In [37]:
len(predict)

459

In [38]:
knn_acc = []
knn_tss = []
knn_prec = []
knn_tn = []
knn_tp = []
knn_fn = []
knn_fp = []

svm_acc = []
svm_tss = []
svm_prec = []
svm_tn = []
svm_tp = []
svm_fn = []
svm_fp = []

randmf_acc = []
randmf_tss = []
randmf_prec = []
randmf_tn = []
randmf_tp = []
randmf_fn = []
randmf_fp = []

for i in range(0,11):

  x_train,x_test,y_train,y_test = train_test_split(X,y,stratify = y , random_state=42,test_size=0.3)
  
  ## Running KNN Algorithm
  knn.fit(x_train,y_train)
  y_pred_knn = knn.predict(x_test)

  tn, fp, fn, tp = confusion_matrix(y_test, y_pred_knn).ravel()
  knn_tn.append(tn)
  knn_tp.append(fp)
  knn_fn.append(fn)
  knn_fp.append(tp)

  acck = (tp + tn) / (tn + fp + fn + tp)
  tss = (tp / (tp - fn)) - (fp / (fp + tn))
  precision = tp / (tp + fp)

  knn_acc.append(acck)
  knn_tss.append(tss)
  knn_prec.append(precision)

  ## Running Random Forest Algorithm
  randomforest.fit(x_train,y_train)
  y_pred_rf = randomforest.predict(x_test)

  tn, fp, fn, tp = confusion_matrix(y_test, y_pred_rf).ravel()
  randmf_tn.append(tn)
  randmf_tp.append(fp)
  randmf_fn.append(fn)
  randmf_fp.append(tp)

  accrf = (tp + tn) / (tn + fp + fn + tp)
  tss = (tp / (tp - fn)) - (fp / (fp + tn))
  precision = tp / (tp + fp)

  randmf_acc.append(accrf)
  randmf_tss.append(tss)
  randmf_prec.append(precision)

  ## Running SVM Algorithm
  classifier.fit(x_train, y_train)
  y_pred_svm = classifier.predict(x_test)

  tn, fp, fn, tp = confusion_matrix(y_test, y_pred_svm).ravel()
  svm_tn.append(tn)
  svm_tp.append(fp)
  svm_fn.append(fn)
  svm_fp.append(tp)

  accs = (tp + tn) / (tn + fp + fn + tp)
  tss = (tp / (tp - fn)) - (fp / (fp + tn))
  precision = tp / (tp + fp)

  svm_acc.append(accs)
  svm_tss.append(tss)
  svm_prec.append(precision)

In [39]:
## Average: 
avg_knn_acc = sum(knn_acc) / len(knn_acc)
avg_knn_tss = sum(knn_tss) / len(knn_tss)
avg_knn_prec = sum(knn_prec) / len(knn_prec)
avg_knn_tn = sum(knn_tn) / len(knn_tn)
avg_knn_tp = sum(knn_tp) / len(knn_tp)
avg_knn_fn = sum(knn_fn) / len(knn_fn)
avg_knn_fp = sum(knn_fp) / len(knn_fp)


In [40]:
## Average: 
avg_svm_acc = sum(svm_acc) / len(svm_acc)
avg_svm_tss = sum(svm_tss) / len(svm_tss)
avg_svm_prec = sum(svm_prec) / len(svm_prec)
avg_svm_tn = sum(svm_tn) / len(svm_tn)
avg_svm_tp = sum(svm_tp) / len(svm_tp)
avg_svm_fn = sum(svm_fn) / len(svm_fn)
avg_svm_fp = sum(svm_fp) / len(svm_fp)

In [41]:
## Average: 
avg_randmf_acc = sum(randmf_acc) / len(randmf_acc)
avg_randmf_tss = sum(randmf_tss) / len(randmf_tss)
avg_randmf_prec = sum(randmf_prec) / len(randmf_prec)
avg_randmf_tn = sum(randmf_tn) / len(randmf_tn)
avg_randmf_tp = sum(randmf_tp) / len(randmf_tp)
avg_randmf_fn = sum(randmf_fn) / len(randmf_fn)
avg_randmf_fp = sum(randmf_fp) / len(randmf_fp)

In [42]:
table_cols = {'Algorithm' : ['KNN' , 'SVM' , 'Random Forest'] , 'TP' : [avg_knn_tp , avg_svm_tp , avg_randmf_tp] , 'FP' : [avg_knn_fp, avg_svm_fp, avg_randmf_fp] , 'FN' : [avg_knn_fn, avg_svm_fn, avg_randmf_fn] , 'TN' : [avg_knn_tn, avg_svm_tn, avg_randmf_tn] , 'ACC' : [avg_knn_acc, avg_svm_acc, avg_randmf_acc], 'TSS' : [avg_knn_tss , avg_svm_tss, avg_randmf_tss] , 'Precision' : [avg_knn_prec, avg_svm_prec, avg_randmf_prec]}


In [43]:
table_cols1 = pd.DataFrame.from_dict(table_cols)

In [44]:
table_cols1

,Algorithm,TP,FP,FN,TN,ACC,TSS,Precision
0,KNN,38.000000,112.000000,41.000000,85.000000,0.713768,1.268522,0.746667
1,SVM,34.000000,113.000000,40.000000,89.000000,0.731884,1.271522,0.768707
2,Random Forest,15.090909,139.818182,13.181818,107.909091,0.897563,0.981563,0.902689
